In [1]:
#!pip install pandas

In [2]:
import json
import pandas as pd
from collections import defaultdict
from itertools import combinations,permutations 
import numpy as np
from sklearn import cluster
import glob,os
import pickle
import warnings
from sklearn.cluster import DBSCAN
warnings.filterwarnings("ignore")

#### Data Processing

In [3]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/"
all_datasets = glob.glob(root + 'split_merged_dataset/*_train.jsonl')
all_datasets
all_datasets[3].split('/')[-1][:-6].split('_')[0]

'ta'

In [4]:
languages = []
for dataset in all_datasets:
    lang = dataset.split('/')[-1][:-6].split('_')[0]
    languages.append(lang)
print(languages)

['pa', 'gu', 'hi', 'ta', 'as', 'ml', 'te', 'kn', 'bn', 'or', 'mr', 'en']


In [5]:
print(all_datasets)

['/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/pa_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/gu_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/hi_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/ta_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/as_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/ml_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/te_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/kn_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/bn_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/or_merged_train.jsonl', '/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/mr_merged_train.jsonl'

In [6]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/"
ent_dict = defaultdict()
fact_dict_all = defaultdict()

languages = []

for subdir, dirs, files in os.walk(root):
    print(subdir)
    
    lang =  subdir.split('/')[-1]
    print(lang)
    
    print(files)
    
    e_dict = {}
    #i=0
    if lang != "":
        with open(subdir+'/train.jsonl') as f:
            unique_facts = []

            for line in f:
#                 if i>10:
#                     break
#                 #i+=1
#                 print(i)
            
                data = json.loads(line)
                f = []
                for l in data['facts_list']:
                    #print('sentence')
                    f2 = []
                    for fact in l:
                        f2.append(fact[0])  
                        # Generate facts 
                        unique_facts.append(fact[0])  
                    f.append(f2)
                    #print(f)
                e_dict[data['entity_name']] = f
                #i+=1
        languages.append(lang)
        fact_dict_all[lang] = set(unique_facts)
        print(lang,len(fact_dict_all[lang]))
        ent_dict[lang] = e_dict

/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/

['add_lang.py']
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/mr
mr
['train.jsonl', 'test.jsonl', 'val.jsonl']
mr 172
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/gu
gu
['train.jsonl', 'test.jsonl', 'val.jsonl']
gu 122
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/te
te
['train.jsonl', 'test.jsonl', 'val.jsonl']
te 151
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/pa
pa
['train.jsonl', 'test.jsonl', 'val.jsonl']
pa 201
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/ml
ml
['train.jsonl', 'test.jsonl', 'val.jsonl']
ml 223
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/kn
kn
['train.jsonl', 'test.jsonl', 'val.jsonl']
kn 150
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/bn
bn
['train.jsonl', 'test.jsonl', 'val.jsonl']
bn 234
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/hi
hi
['train.jsonl', 'test.jsonl', 'val.jsonl']
hi 207
/Users/rahulmehta/Desktop/Mult

In [7]:
#languages

#### Affinity Matrix

In [8]:
# ent_dict = defaultdict()
# fact_dict_all = defaultdict()

# languages = []
# for dataset in all_datasets:
#     lang = dataset.split('/')[-1][:-6].split('_')[0]
    
#     e_dict = {}
#     print(lang)
#     with open(dataset) as f:
        
        
#         unique_facts = []

#         for line in f:
#     #         if i>50:
#     #             break
#             #print(i)
#             data = json.loads(line)
#             f = []
#             for l in data['facts_list']:
#                 #print('sentence')
#                 f2 = []
#                 for fact in l:
#                     f2.append(fact[0])  
#                     # Generate facts 
#                     unique_facts.append(fact[0])  
#                 f.append(f2)
#                 #print(f)
#             e_dict[data['entity_name']] = f
#             i+=1
#     languages.append(lang)
#     fact_dict_all[lang] = set(unique_facts)
#     print(lang,len(fact_dict_all[lang]))
#     ent_dict[lang] = e_dict

In [9]:
#print(languages)

In [10]:
fact_dict = defaultdict()
factsetdict = defaultdict()

for lang in languages:
    print(lang)
    fdict = {i:f for i,f in enumerate(fact_dict_all[lang])}
    
    factsetdict[lang] = set(fact_dict_all[lang])
    
    fact_dict[lang] = fdict
    #print(fact_dict) 

mr
gu
te
pa
ml
kn
bn
hi
as
en
or
ta


In [11]:
perm_dict = defaultdict()
for i,(k,facts) in enumerate(factsetdict.items()):
    perm_dict[k] = list(permutations(facts,2))

#### PMI 

In [12]:
# data=pd.read_csv('../datasets/fact_pair_probs/en.tsv',sep='\t',header=None)
# data.columns = ['relation','scores']
# data.head(5)

In [13]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/"
all_datasets = glob.glob(root + 'fact_pair_probs/*.tsv')
all_datasets 

['/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/ta.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/bn.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/mr.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/te.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/as.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/gu.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/hi.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/en.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/pa.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/kn.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/or.tsv',
 '/Users/rahulmehta/Desktop/MultiSent/datasets/fact_pair_probs/ml.tsv']

In [14]:
languages

['mr', 'gu', 'te', 'pa', 'ml', 'kn', 'bn', 'hi', 'as', 'en', 'or', 'ta']

In [15]:
pmi_dict = defaultdict()
for lang in languages:
    for dataset in all_datasets:
        #print(dataset.split('/')[-1].split('.')[0])
        if lang == dataset.split('/')[-1].split('.')[0]:
            print(lang)
            pmi_list = []
            with open(dataset) as f:
                for line in f:
                    l = line.split('\t')
                    pmi_list.append(l)
    pmi_dict[lang] = pmi_list

mr
gu
te
pa
ml
kn
bn
hi
as
en
or
ta


In [16]:
#pmi_dict

#### Affinity Matrix

In [17]:
# affinity_dict = defaultdict()
# for lang in languages: 
#     print(lang)
#     a_dict = defaultdict()
#     for p in perm_dict[lang]:
#         #print(p)
#         look = p[0] + '|' +  p[1]
#         look2 = p[1] + '|' +  p[0]
#         for item in pmi_dict[lang]:
#             if look == item[0] or look2 == item[0]:
#                 # print("yes")
#                 #print(p)
#                 a_dict[p] = float(item[1].split('\n')[:-1][0])
#                 break
#             else:
#                 a_dict[p] = float(0)
#     affinity_dict[lang] = a_dict

In [18]:
# affinity_dict['en']

#### Create affinity matrix

In [19]:
# for lang in languages:
#     print(lang)
#     #if lang=='en':
#     s = len(fact_dict[lang]),len(fact_dict[lang])
#     affinity_matrix  = np.zeros(s,dtype=float)
#     #print(s)
#     for r in fact_dict[lang].keys():
#         #print(r)
#         cols = np.zeros(len(fact_dict[lang].keys()))
#         for c in fact_dict[lang].keys():
#             for item in affinity_dict[lang].keys():
#                # print(item)
#                 #print(r,item[0])
#                 if fact_dict[lang][r] == item[0] and fact_dict[lang][c] == item[1]:
#                     #print("yes")
#                     #print(affinity_dict[lang][item])
#                     affinity_matrix[r][c] = affinity_dict[lang][item]
#                     affinity_matrix[c][r] = affinity_dict[lang][item]
#     np.save(f'../datasets/affinity_matrices/{lang}_affinity_matrix.npy',affinity_matrix)   
#     print(affinity_matrix.shape)

##### Spectral clustering for all entities on validation set

In [20]:
# root = "/Users/rahulmehta/Desktop/MultiSent/datasets/split_merged_dataset/"
# all_datasets = glob.glob(root  + '*_test.jsonl')
# # all_datasets 

In [21]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/"
ent_dict = defaultdict()
cnts_dict = {}
languages = []


for subdir, dirs, files in os.walk(root):
    print(subdir)
    
    lang =  subdir.split('/')[-1]
    print(lang)
    
    print(files)
    
    e_dict = {}
    cnts = []
    
    if lang != "":
        with open(subdir+'/test.jsonl') as f:
            unique_facts = []

            for line in f:
                c = 0
                data = json.loads(line)
                f = []
                for l in data['facts_list']:
                    #print('sentence')
                    f2 = []
                    for fact in l:
                        f2.append(fact[0])  
                        # Generate facts 
                        unique_facts.append(fact[0])  
                    f.append(f2)
                    #print(f)
                    c+=1

                cnts.append(c)

                e_dict[data['entity_name']] = f

        languages.append(lang)
        #fact_dict_all[lang] = set(unique_facts)
        print(lang,len(fact_dict_all[lang]))
        ent_dict[lang] = e_dict
        cnts_dict[lang] = cnts

/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/

['add_lang.py']
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/mr
mr
['train.jsonl', 'test.jsonl', 'val.jsonl']
mr 172
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/gu
gu
['train.jsonl', 'test.jsonl', 'val.jsonl']
gu 122
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/te
te
['train.jsonl', 'test.jsonl', 'val.jsonl']
te 151
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/pa
pa
['train.jsonl', 'test.jsonl', 'val.jsonl']
pa 201
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/ml
ml
['train.jsonl', 'test.jsonl', 'val.jsonl']
ml 223
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/kn
kn
['train.jsonl', 'test.jsonl', 'val.jsonl']
kn 150
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/bn
bn
['train.jsonl', 'test.jsonl', 'val.jsonl']
bn 234
/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/hi
hi
['train.jsonl', 'test.jsonl', 'val.jsonl']
hi 207
/Users/rahulmehta/Desktop/Mult

In [22]:
cnts_dict_ent = defaultdict()
for i,lang in enumerate(cnts_dict):
    num_sent = pd.Series(cnts_dict[lang]).sum()
    #print(lang,num_sent)
    cnts_dict_ent[lang]=num_sent
print(cnts_dict_ent)

defaultdict(None, {'mr': 1432, 'gu': 415, 'te': 1411, 'pa': 2304, 'ml': 3607, 'kn': 1008, 'bn': 7550, 'hi': 3275, 'as': 485, 'en': 10216, 'or': 982, 'ta': 3731})


In [23]:
pd.Series(cnts_dict['te']).value_counts()

2     76
1     72
3     62
4     48
5     37
6     28
7     21
8     15
9     11
10     9
dtype: int64

##### Clustering on a dataset

In [24]:
# fact_dict = defaultdict()
# factsetdict = defaultdict()

# for lang in languages:
#     print(lang)
#     fdict = {i:f for i,f in enumerate(fact_dict_all[lang])}
    
#     factsetdict[lang] = set(fact_dict_all[lang])
    
#     fact_dict[lang] = fdict
#     #print(fact_dict) 

In [25]:
#fact_dict['en']

In [26]:
#fact_dict['en']

In [27]:
#fact_dict.values()

#### Label Matching on clustering

In [28]:
#fact_dict['en']

#### New logic for cluster matching

In [29]:
#if fact_dict['en'].value

In [30]:
languages

['mr', 'gu', 'te', 'pa', 'ml', 'kn', 'bn', 'hi', 'as', 'en', 'or', 'ta']

In [31]:
#!pip install munkres

In [32]:
from labelmapping import label_matching,make_cost_matrix,translate_clustering,accuracy

### Get number of sentence classifier's predictions 

In [33]:
import pandas as pd
test_all = pd.read_csv('/Users/rahulmehta/Desktop/MultiSent/MEMS-XF2T/dataset_prep/test_all.csv',header=None,names=['text','nsent'],index_col=None) 
test_all['lang']= test_all['text'].astype(str).str[0:2]
test_all['index'] = test_all.index
test_all.head(5)

,text,nsent,lang,index
0,hi <r> occupation <r> country of citizenship,1,hi,0
1,hi <r> occupation <r> field of work,1,hi,1
2,hi <r> sport <r> country of citizenship,1,hi,2
3,hi <r> occupation <r> occupation,1,hi,3
4,hi <r> occupation <r> occupation <r> country o...,1,hi,4


In [34]:
groups = test_all['index'].groupby(test_all['lang'])
res = {lang: list(group) for lang, group in groups}
#res['hi']

In [35]:
# Load predictions from classifier
my_file = open('/Users/rahulmehta/Desktop/MultiSent/MEMS-XF2T/dataset_prep/output.txt')
data = my_file.read()
pred = data.split("\n")[:-1]
#print(pred)
my_file.close()  
pred = [int(i) for i in pred]

In [36]:
pred[0]

2

In [37]:
## Look up predictions
from collections import defaultdict
lang = list(res.keys())
classifier_preds = {l: [] for l in lang}
#print(d1)
for k in res.keys():
    for item in res[k]:
        #print(item)
        classifier_preds[k].append(pred[item])


In [38]:
#classifier_preds['hi']

##### Create lang,line number,and predicted cluster mapping

### Create test dataset for  predicted clusters - one json for one sentence

In [39]:
#ent_dict['en']

In [40]:
#fact_dict['en']

In [42]:
ent_dict_pred = ent_dict
# for e,c in zip(ent_dict.keys(),cnts):
#     print(e,c)
from collections import Counter
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/affinity_matrices/"

entpred_dict_all = defaultdict()

cnt =0
nothing=0
macro_pr = []
macro_rec = []
macro_pr_old = []
macro_rec_old = []
facts_len = 0
weighted_acc =[]
missed = []
scores_dict = defaultdict()

for lang in languages:
    if lang == 'en':

        scores = defaultdict()
        missed = 0

        print(lang)
        affinity_matrix = np.load(root + lang + '_affinity_matrix.npy')
        print("loaded affinity Matrix")
        #print(affinity_matrix.shape)
        

        entpred_dict  = defaultdict()

        for i,v in enumerate(ent_dict[lang].keys()):
            
            cnt+=1
#             if cnt<1000 or cnt>1000:
#                 continue
            print(cnt)
            #print(v,ent_dict[v])
            e = ent_dict[lang][v]
            #c = len(ent_dict[lang][v])
            
            # Get predictions from the prediction csv using line number
            c = classifier_preds[lang][cnt]
            
            

#             print("v",v)
#             print("e",e)
#             print("c",c)

            #print(flat_list)


            # Get uniques facts 

            flat_list = [item for sublist in e for item in sublist]

            flat_set = set(flat_list)
            #print(flat_set)
            flat_ids = []

            for item in flat_set:
                for i,(key,value) in enumerate(fact_dict[lang].items()):
                    if value==item:
                        #print(value)
                        flat_ids.append(key)
            #print(flat_ids)


            # Get ids
            flat_list_cnts = dict(Counter(flat_list))
            #print(flat_list_cnts)

            ent = e
            for i,value in enumerate(ent):
                for j,subitem in enumerate(value):
                    #print(ent[i][j])
                    for lookupitem in flat_list_cnts.keys():
                        #print(lookupitem)
                        if subitem == lookupitem and flat_list_cnts[lookupitem]!=1:    
                            ent[i][j] = tuple([ent[i][j],lookupitem + '_' + str(flat_list_cnts[lookupitem])])
                            flat_list_cnts[lookupitem] -= 1
                            #print(flat_list_cnts)
                            #print(e)
                            continue
                    #print(type(ent[i][j]))
                    if not isinstance(ent[i][j], tuple):
                        #print(True)
                        ent[i][j] = tuple([ent[i][j],ent[i][j]])


            #print("Ent",ent)
            flat_ids = []
            sent_id=0
            act_list = []
            flag=0
            new_fact_id = 200000
            for sent in ent:
                for item in sent:
                    #print("item[0]",item[0])
                    for i,(key,value) in enumerate(fact_dict[lang].items()):
                        if value==item[0]:
                            flag==1
                            flat_ids.append(key)
                            act_list.append(sent_id)
                            break
    #                     flat_ids.append(new_fact_id)
    #                     new_fact_id +=1
    #                break

                sent_id +=1

            #print("Actual Clusters",act_list)
            #print(act_list)

            ent_perm_list = list(permutations(flat_ids,2))
            #print("Flat ids",flat_ids)
           
            #print(fact_dict[lang][flat_ids[0]])

            ent_aff_matrix = np.zeros((len(flat_ids),len(flat_ids)))
            #print(ent_aff_matrix)


            #print(ent_perm_list)
            for p in ent_perm_list:
                #print(affinity_matrix[p[0],p[1]])
                ent_aff_matrix[flat_ids.index(p[0]),flat_ids.index(p[1])] = affinity_matrix[p[0],p[1]]
      
    
            #print(ent_aff_matrix)
            # clusters
            try:
                #print("num cluster",c)
                ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
                #print("spectral Clusters",ent_clusters)
                
#                 clustering = DBSCAN(eps=0.05, min_samples=1,leaf_size=1,p=1,algorithm='auto').fit(ent_aff_matrix)#.fit_predict()
#                 ent_clusters = DBSCAN(eps=0.05, min_samples=1,leaf_size=1,p=1,algorithm='auto').fit_predict(ent_aff_matrix)
#                 print("dbscan Clusters",ent_clusters)
#                 print("dbscan_nclusters",len(np.unique(ent_clusters)))
            
                
            except Exception:
                #nothing +=1
                #print(nothing)
                #print("No clusters")
                #print(ent_aff_matrix.shape,c)
                continue

            
            # Number of predicted clusters
            #c = len(np.unique(clustering.labels_))
            #print("No of predicted cl",len(np.unique(clustering.labels_)))
            # Assign predicted facts to predicted clusters   
            
            pred_facts = [[] for i in range(c)]
            for index,fid in enumerate(flat_ids):                
                fact_name = fact_dict[lang][fid]
                pred_cluster = ent_clusters[index]
                #print(fid,fact_name,pred_cluster)
                pred_facts[pred_cluster].append(fact_name)
            #print(pred_facts)
            
            ent_dict_pred[lang][v] = pred_facts
            #print(e)
            print(pred_facts)
            # LABEL MAPPING
#             if (pd.Series(act_list).nunique() == pd.Series(ent_clusters).nunique()):
#                 new_clusters,cm,old_acc,old_pr,old_re,new_cm,new_acc,new_pr,new_re  = label_matching(act_list,ent_clusters)
#             else:
#                 #print("Missed entity")
#                 missed +=1
#                 print("missed"
#http://localhost:8888/notebooks/src/01-spectral_clustering_main-classifier%2Bclustering.ipynb#
#             entpred_dict[v] = new_clustexrs

en
loaded affinity Matrix
1
[['date of death', 'date of birth'], ['occupation']]
2
[['date of birth', 'occupation'], ['country of citizenship']]
3
[['date of birth', 'occupation'], ['date of death']]
4
[['country of citizenship'], ['occupation']]
5
[['date of birth'], ['place of birth']]
6
7
[['occupation', 'date of birth'], ['occupation']]
8
[['country of citizenship', 'occupation'], ['place of birth', 'occupation']]
9
[['occupation', 'occupation'], ['date of birth', 'sport']]
10
[['country of citizenship'], ['place of birth']]
11
12
[['occupation', 'sport'], ['date of birth', 'occupation']]
13
[['place of birth'], ['date of birth']]
14
[['date of death', 'date of birth'], ['occupation']]
15
[['sport'], ['date of birth', 'occupation']]
16
[['date of birth', 'occupation'], ['sport']]
17
[['date of birth'], ['occupation']]
18
[['date of birth'], ['occupation', 'country of citizenship']]
19
[['date of birth'], ['occupation']]
20
[['occupation'], ['place of birth', 'occupation', 'date of 

[['date of death'], ['member of political party']]
183
[['occupation', 'date of birth'], ['occupation']]
184
[['occupation'], ['date of birth']]
185
186
[['date of death'], ['date of birth', 'occupation']]
187
[['date of birth'], ['occupation']]
188
[['floruit', 'languages spoken, written or signed'], ['occupation']]
189
[['date of death', 'occupation', 'date of birth'], ['country of citizenship']]
190
[['date of birth', 'country of citizenship'], ['place of birth', 'occupation']]
191
[['date of death', 'member of sports team'], ['date of birth', 'occupation']]
192
193
[['date of death'], ['country of citizenship']]
194
[['date of birth', 'occupation'], ['place of birth', 'member of sports team']]
195
[['member of sports team', 'occupation'], ['country of citizenship']]
196
[['occupation'], ['country of citizenship']]
197
198
199
200
[['place of birth'], ['educated at']]
201
[['place of death'], ['date of death', 'date of birth', 'languages spoken, written or signed', 'place of birth']

[['occupation', 'date of birth', 'member of sports team'], ['place of birth']]
340
[['occupation'], ['date of birth']]
341
[['date of birth'], ['occupation']]
342
[['country of citizenship', 'member of', 'occupation', 'languages spoken, written or signed'], ['date of birth', 'occupation']]
343
[['date of birth', 'occupation'], ['country of citizenship', 'member of sports team', 'place of birth']]
344
[['sport'], ['date of death', 'occupation', 'member of sports team', 'date of birth', 'occupation']]
345
[['occupation'], ['relative', 'relative']]
346
[['occupation'], ['country of citizenship']]
347
[['sport', 'date of birth', 'occupation'], ['country of citizenship']]
348
[['date of birth', 'occupation'], ['country of citizenship']]
349
[['place of death'], ['date of death']]
350
[['member of sports team', 'occupation', 'country of citizenship'], ['date of birth']]
351
[['country of citizenship'], ['date of birth', 'occupation']]
352
[['date of birth'], ['country of citizenship']]
353
[

474
[['date of death', 'date of birth', 'country of citizenship', 'field of work'], ['occupation', 'place of birth']]
475
[['occupation', 'member of political party', 'place of death'], ['date of death', 'date of birth']]
476
[['date of birth'], ['occupation']]
477
[['named after', 'date of death', 'occupation', 'date of death', 'country of citizenship', 'field of work'], ['date of birth', 'occupation']]
478
[['date of birth', 'country of citizenship', 'occupation', 'occupation'], ['occupation']]
479
[['occupation', 'date of birth'], ['country of citizenship']]
480
[['date of birth', 'occupation'], ['place of birth']]
481
[['occupation'], ['date of birth', 'country of citizenship']]
482
[['occupation'], ['position held']]
483
[['place of birth'], ['date of birth']]
484
[['date of birth'], ['occupation']]
485
[['occupation', 'country of citizenship'], ['position held']]
486
487
[['place of birth', 'country of citizenship', 'occupation'], ['record label', 'instrument', 'languages spoken,

[['sibling'], ['occupation']]
637
[['occupation', 'occupation'], ['date of birth', 'date of death']]
638
[['date of birth', 'occupation'], ['participant in', 'participant in', 'sport', 'country of citizenship']]
639
[['occupation'], ['date of birth', 'sport', 'country of citizenship']]
640
[['occupation'], ['date of birth']]
641
642
[['occupation'], ['date of birth', 'country of citizenship']]
643
[['date of death', 'occupation'], ['participant in', 'participant in', 'date of birth', 'competition class']]
644
645
[['occupation'], ['member of sports team', 'sibling']]
646
[['date of birth'], ['occupation']]
647
[['occupation'], ['occupation']]
648
[['languages spoken, written or signed'], ['cast member', 'occupation', 'country of citizenship']]
649
[['occupation', 'date of birth'], ['place of birth']]
650
[['country of citizenship', 'occupation'], ['position held']]
651
[['occupation', 'occupation'], ['sibling']]
652
653
[['occupation', 'occupation'], ['discography', 'date of birth']]
6

...

[['occupation', 'date of birth', 'record held', 'record held', 'record held', 'record held', 'sports discipline competed in', 'record held', 'occupation', 'winner', 'winner', 'award received', 'award received', 'winner', 'award received'], ['record held', 'award received', 'winner', 'educated at']]
4101
[['occupation', 'birth name', 'date of birth', 'occupation', 'place of birth', 'country of citizenship', 'award received', 'winner', 'member of sports team', 'award received', 'award received'], ['sport', 'winner', 'country of citizenship', 'occupation', 'occupation']]
4102
[['employer', 'award received', 'award received', 'award received', 'award received', 'member of', 'member of', 'award received', 'member of', 'award received', 'award received', 'award received', 'award received', 'award received', 'member of'], ['place of birth', 'member of']]
4103
[['place of birth'], ['field of work', 'occupation', 'date of death', 'military branch', 'military rank', 'date of birth', 'military br

[['occupation', 'date of birth', 'occupation', 'occupation', 'nominated for', 'award received', 'nominated for', 'director', 'cast member', 'nominated for', 'nominated for', 'award received', 'director', 'award received', 'date of death'], ['educated at', 'screenwriter', 'languages spoken, written or signed', 'cause of death']]
4150
[['date of birth', 'occupation', 'nominated for', 'cast member', 'award received', 'educated at', 'educated at', 'spouse'], ['mother', 'place of birth']]
4151
[['position held', 'field of work', 'field of work', 'educated at', 'field of work', 'occupation', 'employer', 'occupation', 'award received'], ['educated at', 'employer']]
4152
[['position held', 'position held', 'position held', 'position held', 'position held', 'position held', 'date of birth', 'place of birth', 'country of citizenship', 'position held', 'position held', 'educated at', 'position held', 'position held', 'position held', 'position held', 'position held', 'position held', 'occupation'

[['date of birth', 'occupation', 'occupation', 'occupation', 'languages spoken, written or signed', 'occupation', 'award received', 'educated at', 'educated at', 'educated at', 'member of', 'member of', 'member of', 'member of', 'named after', 'award received', 'occupation'], ['date of death', 'named after', 'member of']]
4187
[['occupation', 'position held', 'member of political party', 'military branch', 'position held', 'director / manager', 'country of citizenship', 'date of birth', 'military rank', 'candidacy in election', 'residence'], ['position held', 'date of birth', 'residence', 'place of birth', 'position held', 'position held']]
4188
[['birth name', 'date of birth', 'nominated for', 'nominated for', 'cast member', 'award received', 'nominated for', 'nominated for', 'nominated for', 'award received', 'nominated for'], ['filmography', 'nominated for', 'nominated for', 'nominated for', 'nominated for', 'nominated for', 'nominated for', 'cast member', 'nominated for', 'nominate

[['occupation', 'date of birth', 'award received', 'award received', 'occupation', 'award received', 'composer', 'occupation', 'composer', 'composer', 'award received', 'award received', 'occupation', 'award received', 'award received', 'winner', 'composer', 'award received', 'occupation', 'award received', 'occupation', 'award received', 'award received'], ['occupation', 'award received', 'award received', 'award received', 'award received', 'award received', 'award received', 'award received', 'award received', 'occupation', 'award received', 'award received', 'performer', 'child', 'child', 'award received', 'performer', 'award received', 'award received']]
4233
[['military branch', 'military rank', 'date of birth', 'date of death', 'occupation', 'occupation', 'award received', 'award received', 'award received', 'award received', 'member of', 'award received', 'award received', 'award received', 'award received', 'award received', 'award received', 'member of', 'award received', 'aw

[['occupation', 'award received', 'award received', 'award received', 'place of birth', 'winner', 'occupation', 'cast member', 'award received', 'award received', 'cast member', 'nominated for', 'cast member'], ['date of birth', 'occupation', 'award received', 'educated at', 'cast member', 'occupation']]
4268
[['named after', 'father', 'family', 'position held', 'present in work', 'position held', 'present in work', 'present in work', 'founded by', 'present in work', 'spouse', 'place of death', 'spouse'], ['date of death']]
4269
[['position held', 'name in native language', 'educated at'], ['date of birth', 'occupation', 'position held', 'position held', 'position held', 'position held', 'member of political party', 'place of birth', 'position held', 'position held', 'position held', 'educated at', 'spouse', 'member of political party']]
4270
[['date of birth', 'country of citizenship', 'head of government', 'participant', 'educated at', 'position held'], ['position held', 'head of sta

In [ ]:
#ent_dict_pred

In [ ]:
#ent_dict_pred['en']["Alexei Abrikosov"]

In [ ]:
#!--NotebookApp.iopub_data_rate_limit=3000000.0 

In [ ]:
#ent_dict_pred['en']['Balbir Singh, Sr.']

In [56]:
l = [[['date of death', '26 September 1958', [], False], ['date of birth', '01 April 1891', [], False], ['occupation', 'academic', [], False]]]
for i in l:
    for j in i:
         print(j)

['date of death', '26 September 1958', [], False]
['date of birth', '01 April 1891', [], False]
['occupation', 'academic', [], False]


In [88]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/split_data/"
pred_dict_final = defaultdict()
cnts_dict = {}
languages = []
output_path = "/Users/rahulmehta/Desktop/MultiSent/datasets/split_data_clusters_classifier"

for subdir, dirs, files in os.walk(root):
    lang =  subdir.split('/')[-1]
    print(lang)    
     
#     if lang != "":
#         print(lang)
#         i = 0
    i = 0 
    if lang == "en":
        with open(subdir+'/test.jsonl') as f:
            pred_list_all = []

            unique_facts = []

            for line in f:

                i+=1
                if i < 1000 or i > 1000:
                    continue
                print(i)

                #print(i)   
                #print(line)
                pred_final = {}

                data = json.loads(line)
                #print(data)
                f = []
                data['facts'] = ent_dict_pred[lang][data['entity_name']]
                pred_final["entity_name"] = data["entity_name"]
                #pred_final["sentence_list"] = data["sentence_list"]
                pred_final['facts_list'] = data["facts_list"]
                pred_final["facts"] = ent_dict_pred[lang][data["entity_name"]] 
#                 pred_final["qid"]= data["qid"]
#                 pred_final["native_sentence_section_list"]= data["native_sentence_section_list"]
#                 pred_final["translated_sentence_list"]= data["translated_sentence_list"]
#                 pred_final["sent_index_list"]= data["sent_index_list"]
#                 pred_final["sent_score_list"]= data["sent_score_list"]
#                 pred_final["translated_facts_list"]= data["translated_facts_list"]
#                 pred_final["coverage_score_list"]= data["coverage_score_list"]
#                 pred_final["order_list"]= data["order_list"]
#                 pred_final["role_spec_order_list"]= data["role_spec_order_list"]
#                 pred_final["source_list"]= data["source_list"]
#                 pred_final["sent_label"]= data["sent_label"]
#                 pred_final["avg_coverage"]= data["avg_coverage"]
#                 pred_final["coherence_scores_list"]= data["coherence_scores_list"]
#                 pred_final["avg_coherence"]= data["avg_coherence"]
#                 pred_final["norm_cov"]= data["norm_cov"]
#                 pred_final["norm_coh"]= data["norm_coh"]
#                 pred_final["compiled_score"]= data["compiled_score"]
#                 pred_final["lang"]= data["lang"]


                # Split by sentences
                d = pred_final
                #print(d['facts'])                
            
                #l = len(d['sentence_list'])
                # Update based on number of facts list
                l = len(d['facts'])
                
                #print('l ',l)

#                 dict_list = []
#                 for item in range(l):
#                     dict_list.append(d.copy())
#                 print(len(dict_list))
                f_list = []
                for facts in d['facts']:
                    inner_list = []
                    for f in facts:
                        #print(item)
                        flag=0
                        for it in d['facts_list']:
                            for l in it:
                                #print(l)
                                if l[0] == f:
                                    #print("appended fact")
                                    #print(l)
                                    inner_list.append(l)
                                    flag=1
                                    break
                            if flag==1:
                                break
                    f_list.append(inner_list)
#                     item['facts'] = f_list 
                #print("f_list",f_list)
                    
        
                pred_list_all = []
                for item in f_list:
                    #print(item)
                    new_dict = pred_final.copy()
                    new_dict['facts'] = item
                    pred_list_all.append(new_dict)

                #print(pred_list_all)
        if not os.path.exists(os.path.join(output_path,lang)): 
            os.mkdir(os.path.join(output_path,lang))

        #print(len(pred_list_all))
        with open(output_path  + '/'+ lang + '/test.jsonl','a') as fp: 
            for item in pred_list_all:
                #print(i)
                json.dump(item,fp,ensure_ascii=False)
                fp.write('\n')
            print("file written")


mr
gu
te
pa
ml
kn
bn
hi
as
en
1000
file written
or
ta


In [ ]:
d={'facts_list': [[['date of death', '10 October 1995', [], False], ['date of birth', '04 January 1923', [], False], ['occupation', 'politician', [], False], ['spouse', 'Damodara Menon', [], False]], [['member of political party', 'Indian National Congress', [], False], ['occupation', 'politician', [], False]]]}
ll = d['facts_list']
f = 'date of death'
l = []
for it in ll:
    for i in it:
        if i[0] == f:
            l.append(i)
print(l)

In [ ]:
d = {'a':1,'b':[2,3,4],'c':5}
d = pred_final
l = len(d['b'])


dict_list = []
for i in range(l):
    dict_list.append(d.copy())
    
   
for index,item in enumerate(dict_list):
    item['b'] = item['b'][index]
    print(dict_list)


#### Create test dataset ends - skip the above if not required

In [ ]:
#ent_dict 
# for e,c in zip(ent_dict.keys(),cnts):
#     print(e,c)
from collections import Counter
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/affinity_matrices/"

entpred_dict_all = defaultdict()

cnt =0
nothing=0
macro_pr = []
macro_rec = []
macro_pr_old = []
macro_rec_old = []
facts_len = 0
weighted_acc =[]
missed = []
scores_dict = defaultdict()

for lang in languages:
    if lang == 'en':

        scores = defaultdict()
        missed = 0

        print(lang)
        affinity_matrix = np.load(root + lang + '_affinity_matrix.npy')
        print("loaded affinity Matrix")
        print(affinity_matrix.shape)

        entpred_dict  = defaultdict()

        for i,v in enumerate(ent_dict[lang].keys()):
            cnt+=1
            if cnt>1000:
                break
            print(i)
            #print(v,ent_dict[v])
            e = ent_dict[lang][v]
            c = len(ent_dict[lang][v])

            print("v",v)
            print("e",e)
            print("c",c)

            #print(flat_list)


            # Get uniques facts 

            flat_list = [item for sublist in e for item in sublist]

            flat_set = set(flat_list)
            #print(flat_set)
            flat_ids = []

            for item in flat_set:
                for i,(key,value) in enumerate(fact_dict[lang].items()):
                    if value==item:
                        #print(value)
                        flat_ids.append(key)
            #print(flat_ids)


            # Get ids
            flat_list_cnts = dict(Counter(flat_list))
            #print(flat_list_cnts)

            ent = e
            for i,value in enumerate(ent):
                for j,subitem in enumerate(value):
                    #print(ent[i][j])
                    for lookupitem in flat_list_cnts.keys():
                        #print(lookupitem)
                        if subitem == lookupitem and flat_list_cnts[lookupitem]!=1:    
                            ent[i][j] = tuple([ent[i][j],lookupitem + '_' + str(flat_list_cnts[lookupitem])])
                            flat_list_cnts[lookupitem] -= 1
                            #print(flat_list_cnts)
                            #print(e)
                            continue
                    #print(type(ent[i][j]))
                    if not isinstance(ent[i][j], tuple):
                        #print(True)
                        ent[i][j] = tuple([ent[i][j],ent[i][j]])


            #print("Ent",ent)
            flat_ids = []
            sent_id=0
            act_list = []
            flag=0
            new_fact_id = 200000
            for sent in ent:
                for item in sent:
                    #print("item[0]",item[0])
                    for i,(key,value) in enumerate(fact_dict[lang].items()):
                        if value==item[0]:
                            flag==1
                            flat_ids.append(key)
                            act_list.append(sent_id)
                            break
    #                     flat_ids.append(new_fact_id)
    #                     new_fact_id +=1
    #                break

                sent_id +=1

    #        print("Actual Clusters",act_list)
    #         print(act_list)

            ent_perm_list = list(permutations(flat_ids,2))


            ent_aff_matrix = np.zeros((len(flat_ids),len(flat_ids)))
            #print(ent_aff_matrix.shape)


            #print(ent_perm_list)
            for p in ent_perm_list:
                ent_aff_matrix[flat_ids.index(p[0]),flat_ids.index(p[1])] = affinity_matrix[p[0],p[1]]

            #print(ent_aff_matrix)


            # clusters
            try:
                #ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
                #print("Clusters",ent_clusters)
                ent_clusters = cluster.spectral_clustering(affinity=ent_aff_matrix,n_clusters=c)#.fit_predict()
                print("predicted Clusters",ent_clusters)
            except Exception:
                #nothing +=1
                #print(nothing)
                #print("No clusters")
                #print(ent_aff_matrix.shape,c)
                continue


            # LABEL MAPPING
            if (pd.Series(act_list).nunique() == pd.Series(ent_clusters).nunique()):
                new_clusters,cm,old_acc,old_pr,old_re,new_cm,new_acc,new_pr,new_re  = label_matching(act_list,ent_clusters)
            else:
                #print("Missed entity")
                missed +=1
                print("missed")

            #print("New Clusters",new_clusters)

            macro_pr.append(new_pr)
            macro_rec.append(new_re)


            # Weighted accuracy
            #print(new_acc)
            weighted_acc.append(new_acc * len(flat_ids))
            facts_len+= len(flat_ids)

            macro_pr_old.append(old_pr)
            macro_rec_old.append(old_re)


            entpred_dict[v] = new_clusters

        scores['missed_ent'] = missed
        macro_pr_total = pd.Series(macro_pr).sum()/cnt
        macro_rec_total = pd.Series(macro_rec).sum()/cnt
        weighted_acc_total = pd.Series(weighted_acc).sum()/facts_len
        macro_pr_old_total = pd.Series(macro_pr_old).sum()/cnt
        macro_rec_old_total = pd.Series(macro_rec_old).sum()/cnt

        scores['macro_pr'] = round(macro_pr_total*100,2)
        scores['macro_rec'] = round(macro_rec_total*100,2)
        scores['weighted_acc'] = round(weighted_acc_total*100,2)
        scores['macro_pr_old'] = round(macro_pr_old_total*100,2)
        scores['macro_rec_old'] = round(macro_rec_old_total*100,2)
        print(scores)
        scores_dict[lang] = scores

In [ ]:
scores_dict

In [ ]:
cols  =['language','total_ent','total_sent_all','macro_pr','macro_rec','weighted_acc','macro_pr_old','macro_rec_old']
df = pd.DataFrame(columns=cols)

for i,lang in enumerate(scores_dict):
    #print(lang)
    df2 = [{'language': lang, 'total_ent':len(ent_dict[lang]),
            'total_sent_all':cnts_dict_ent[lang],
#           'total_sent':scores_dict[lang]['total_sent'],
#            'total_sent_cov':round(scores_dict[lang]['total_sent']/cnts_dict_ent[lang],2),
        'missed_ent':scores_dict[lang]['missed_ent'],
       'macro_pr':scores_dict[lang]['macro_pr'],
        'macro_rec':scores_dict[lang]['macro_rec'],
        'weighted_acc':scores_dict[lang]['weighted_acc']
        ,'macro_pr_old':scores_dict[lang]['macro_pr_old']
        ,'macro_rec_old':scores_dict[lang]['macro_rec_old']}]
    #print(df2)
    df2= pd.DataFrame.from_dict(df2,orient='columns')
    #print(df2)
    
    df = pd.concat([df,df2], ignore_index=True)
    #df.append(df2)
    
print(df)    

df.to_csv("../results/clustering-precisionrecall-alllang-24Jan2023.csv",index=False)

In [ ]:
# macro_pr = pd.Series(macro_pr).sum()/cnt
# macro_rec = pd.Series(macro_rec).sum()/cnt
# weighted_acc = pd.Series(weighted_acc).sum()/facts_len
# macro_pr_old = pd.Series(macro_pr_old).sum()/cnt
# macro_rec_old = pd.Series(macro_rec_old).sum()/cnt
# print("Macro Precision",(macro_pr,macro_pr_old))
# print("Macro Recall",(macro_rec,macro_rec_old))
# print("Weighted_acc",weighted_acc)

##### Store validation cluster predictions

In [ ]:
#entpred_dict_all['te']

In [ ]:
entpred_dict_all.keys()

In [ ]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/cluster_predictions/"
with open(root + 'cluster_predictions_all.pkl', 'wb') as f:
    pickle.dump(entpred_dict_all, f)

##### Accuracy on validation set

In [ ]:
#len(entpred_dict_all['en'])

In [ ]:
scores_dict

In [ ]:
cols  =['language','total_ent','total_sent_all','macro_pr','macro_rec','weighted_acc','macro_pr_old','macro_rec_old']
df = pd.DataFrame(columns=cols)

for i,lang in enumerate(scores_dict):
    #print(lang)
    df2 = [{'language': lang, 'total_ent':len(ent_dict[lang]),
            'total_sent_all':cnts_dict_ent[lang],
#           'total_sent':scores_dict[lang]['total_sent'],
#            'total_sent_cov':round(scores_dict[lang]['total_sent']/cnts_dict_ent[lang],2),
        'missed_ent':scores_dict[lang]['missed_ent'],
       'macro_pr':scores_dict[lang]['macro_pr'],
        'macro_rec':scores_dict[lang]['macro_rec'],
        'weighted_acc':scores_dict[lang]['weighted_acc']
        ,'macro_pr_old':scores_dict[lang]['macro_pr_old']
        ,'macro_rec_old':scores_dict[lang]['macro_rec_old']}]
    #print(df2)
    df2= pd.DataFrame.from_dict(df2,orient='columns')
    #print(df2)
    
    df = pd.concat([df,df2], ignore_index=True)
    #df.append(df2)
    
print(df)    

df.to_csv("../results/clustering-precisionrecall-alllang-24Jan2023.csv",index=False)

In [ ]:
df 

In [ ]:
df.to_csv("../results/clustering-scores-alllang-24Jan2022.csv",index=False)